In [1]:
def mul2(t0, t1):
    tmp = t0
    t0 = t1
    t1 = tmp ^ t0
    return t0, t1


In [2]:
def mds(state):
    # state = [x0, x1, x2, x3, x4, x5, x6, x7]
    x0, x1, x2, x3, x4, x5, x6, x7 = state

    # local XORs
    x4 ^= x6
    x5 ^= x7
    x0 ^= x2
    x1 ^= x3

    # small 2×2 mixing on pairs
    x2, x3 = mul2(x2, x3)
    x6, x7 = mul2(x6, x7)

    # cross mixing
    x2 ^= x4
    x3 ^= x5
    x6 ^= x0
    x7 ^= x1

    # double MUL2 on A and C
    x0, x1 = mul2(*mul2(x0, x1))
    x4, x5 = mul2(*mul2(x4, x5))

    # final XORs
    x4 ^= x6
    x5 ^= x7
    x0 ^= x2
    x1 ^= x3
    x2 ^= x4
    x3 ^= x5
    x6 ^= x0
    x7 ^= x1

    return [x0, x1, x2, x3, x4, x5, x6, x7]


In [3]:
def mds_influence(mds_func):
    """
    Compute 8x8 binary influence matrix for your toy MDS.
    A[i][j] = 1 if output position i can be affected by input position j.
    """
    N = 8
    A = [[0]*N for _ in range(N)]

    for j in range(N):
        state = [0]*N
        state[j] = 0x1  # any non-zero 4-bit value is fine
        out = mds_func(state.copy())

        for i in range(N):
            if out[i] != 0:
                A[i][j] = 1

    return A

A = mds_influence(mds)
print("Influence matrix A:")
for row in A:
    print(row)


Influence matrix A:
[1, 1, 1, 0, 1, 0, 1, 0]
[1, 0, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 0, 0]
[0, 1, 1, 0, 1, 1, 0, 0]
[1, 0, 1, 0, 1, 1, 1, 0]
[0, 1, 0, 1, 1, 0, 0, 1]
[0, 1, 0, 0, 1, 0, 1, 1]
[1, 1, 0, 0, 0, 1, 1, 0]


In [4]:
import gurobipy as gp
from gurobipy import GRB

N = 8  # 8 logical positions / registers

# A from above
A = mds_influence(mds)

# --- Build MILP model ---
m = gp.Model("ToySaturnin_MILP_v2")

# Binary vars: activity of each logical nibble/register
x_in   = m.addVars(N, vtype=GRB.BINARY, name="x_in")
x_sbox = m.addVars(N, vtype=GRB.BINARY, name="x_sbox")
x_mds  = m.addVars(N, vtype=GRB.BINARY, name="x_mds")

# Objective: minimize number of active S-boxes
m.setObjective(gp.quicksum(x_sbox[i] for i in range(N)), GRB.MINIMIZE)

# 1️⃣ S-box propagation: input → S-box output
for i in range(N):
    m.addConstr(x_sbox[i] >= x_in[i], name=f"SBOX_{i}")

# 2️⃣ MDS diffusion using *real* influence matrix A
for i in range(N):
    # all inputs that affect y_i according to A
    preds = [j for j in range(N) if A[i][j] == 1]
    k = len(preds)
    if k > 0:
        # k * x_mds[i] ≥ sum_j x_sbox[j], j in preds
        m.addConstr(
            k * x_mds[i] >= gp.quicksum(x_sbox[j] for j in preds),
            name=f"MDS_{i}"
        )

# 3️⃣ Non-trivial input (avoid all-zero differential)
m.addConstr(gp.quicksum(x_in[i] for i in range(N)) >= 1, name="Nontrivial")

# --- Solve ---
m.optimize()

print("\n=== MILP v2 Results ===")
for i in range(N):
    print(f"x_in[{i}]   = {int(x_in[i].X)}")
for i in range(N):
    print(f"x_sbox[{i}] = {int(x_sbox[i].X)}")
for i in range(N):
    print(f"x_mds[{i}]  = {int(x_mds[i].X)}")

print(f"\nTotal active S-boxes = {m.objVal}")


Set parameter Username


Set parameter LicenseID to value 2734209


Academic license - for non-commercial use only - expires 2026-11-07


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")


CPU model: 12th Gen Intel(R) Core(TM) i5-12450HX, instruction set [SSE2|AVX|AVX2]


...

Thread count was 1 (of 12 available processors)


Solution count 2: 1 8 


Optimal solution found (tolerance 1.00e-04)


Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%



=== MILP v2 Results ===
x_in[0]   = 0
x_in[1]   = 0
x_in[2]   = 0
x_in[3]   = 0
x_in[4]   = 0
x_in[5]   = 0
x_in[6]   = 0
x_in[7]   = 1
x_sbox[0] = 0
x_sbox[1] = 0
x_sbox[2] = 0
x_sbox[3] = 0
x_sbox[4] = 0
x_sbox[5] = 0
x_sbox[6] = 0
x_sbox[7] = 1
x_mds[0]  = 1
x_mds[1]  = 1
x_mds[2]  = 1
x_mds[3]  = 1
x_mds[4]  = 1
x_mds[5]  = 1
x_mds[6]  = 1
x_mds[7]  = 1

Total active S-boxes = 1.0


In [5]:
import gurobipy as gp
from gurobipy import GRB

N = 8  # 8 positions

# === Model ===
m = gp.Model("ToySaturnin_1round_tightMDS")

# Variables
x_in   = m.addVars(N, vtype=GRB.BINARY, name="x_in")
x_sbox = m.addVars(N, vtype=GRB.BINARY, name="x_sbox")
x_mds  = m.addVars(N, vtype=GRB.BINARY, name="x_mds")

# Objective: minimize active S-boxes (round 0)
m.setObjective(gp.quicksum(x_sbox[i] for i in range(N)), GRB.MINIMIZE)

# 1) S-box propagation: input -> after S-box
for i in range(N):
    m.addConstr(x_sbox[i] >= x_in[i], name=f"SBOX_0_{i}")

# 2) Tight MDS modeling using A
for i in range(N):
    preds = [j for j in range(N) if A[i][j] == 1]
    if not preds:
        # no predecessors => x_mds[i] must be 0
        m.addConstr(x_mds[i] == 0, name=f"MDS_zero_{i}")
        continue

    k = len(preds)

    # lower bound: if any pred active => x_mds[i] must be 1
    m.addConstr(k * x_mds[i] >= gp.quicksum(x_sbox[j] for j in preds),
                name=f"MDS_lb_{i}")

    # upper bound: if no preds active => x_mds[i] must be 0
    m.addConstr(x_mds[i] <= gp.quicksum(x_sbox[j] for j in preds),
                name=f"MDS_ub_{i}")

# 3) Non-trivial input: avoid all-zero pattern
m.addConstr(gp.quicksum(x_in[i] for i in range(N)) >= 1, name="Nontrivial_0")

m.optimize()

print("\n=== 1-round MILP Results (tight MDS) ===")
for i in range(N):
    print(f"x_in[{i}]   = {int(x_in[i].X)}")
for i in range(N):
    print(f"x_sbox[{i}] = {int(x_sbox[i].X)}")
for i in range(N):
    print(f"x_mds[{i}]  = {int(x_mds[i].X)}")

print("\nTotal active S-boxes (round 0) =", m.objVal)


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")


CPU model: 12th Gen Intel(R) Core(TM) i5-12450HX, instruction set [SSE2|AVX|AVX2]


Thread count: 12 physical cores, 12 logical processors, using up to 12 threads


Optimize a model with 25 rows, 24 columns and 108 nonzeros


Model fingerprint: 0x94c63c4f


...

Thread count was 1 (of 12 available processors)


Solution count 2: 1 8 


Optimal solution found (tolerance 1.00e-04)


Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%



=== 1-round MILP Results (tight MDS) ===
x_in[0]   = 0
x_in[1]   = 0
x_in[2]   = 0
x_in[3]   = 0
x_in[4]   = 0
x_in[5]   = 0
x_in[6]   = 0
x_in[7]   = 1
x_sbox[0] = 0
x_sbox[1] = 0
x_sbox[2] = 0
x_sbox[3] = 0
x_sbox[4] = 0
x_sbox[5] = 0
x_sbox[6] = 0
x_sbox[7] = 1
x_mds[0]  = 0
x_mds[1]  = 1
x_mds[2]  = 0
x_mds[3]  = 0
x_mds[4]  = 0
x_mds[5]  = 1
x_mds[6]  = 1
x_mds[7]  = 0

Total active S-boxes (round 0) = 1.0


In [6]:
import gurobipy as gp
from gurobipy import GRB

N = 8
ROUNDS = 2  # 0 and 1

# === Model ===
m2 = gp.Model("ToySaturnin_2round_tightMDS")

# Variables indexed by round r and position i
x_in   = m2.addVars(ROUNDS, N, vtype=GRB.BINARY, name="x_in")
x_sbox = m2.addVars(ROUNDS, N, vtype=GRB.BINARY, name="x_sbox")
x_mds  = m2.addVars(ROUNDS, N, vtype=GRB.BINARY, name="x_mds")

# Objective: total active S-boxes over all rounds
m2.setObjective(
    gp.quicksum(x_sbox[r, i] for r in range(ROUNDS) for i in range(N)),
    GRB.MINIMIZE
)

# --- For each round ---
for r in range(ROUNDS):

    # 1) S-box propagation in round r
    for i in range(N):
        m2.addConstr(x_sbox[r, i] >= x_in[r, i], name=f"SBOX_{r}_{i}")

    # 2) Tight MDS in round r using A
    for i in range(N):
        preds = [j for j in range(N) if A[i][j] == 1]
        if not preds:
            m2.addConstr(x_mds[r, i] == 0, name=f"MDS_zero_{r}_{i}")
            continue

        k = len(preds)

        # lower bound: any active pred forces x_mds[r,i] = 1
        m2.addConstr(k * x_mds[r, i] >= gp.quicksum(x_sbox[r, j] for j in preds),
                     name=f"MDS_lb_{r}_{i}")

        # upper bound: no active preds => x_mds[r,i] = 0
        m2.addConstr(x_mds[r, i] <= gp.quicksum(x_sbox[r, j] for j in preds),
                     name=f"MDS_ub_{r}_{i}")

# 3) Round linkage: output of round 0 becomes input of round 1
for i in range(N):
    m2.addConstr(x_in[1, i] == x_mds[0, i], name=f"Link_r0_to_r1_{i}")

# 4) Non-trivial input only for the first round input
m2.addConstr(gp.quicksum(x_in[0, i] for i in range(N)) >= 1, name="Nontrivial_input")

m2.optimize()

print("\n=== 2-round MILP Results (tight MDS) ===")
for r in range(ROUNDS):
    print(f"\n--- Round {r} ---")
    for i in range(N):
        print(f"x_in[{r},{i}]   = {int(x_in[r, i].X)}")
    for i in range(N):
        print(f"x_sbox[{r},{i}] = {int(x_sbox[r, i].X)}")
    for i in range(N):
        print(f"x_mds[{r},{i}]  = {int(x_mds[r, i].X)}")

print("\nTotal active S-boxes over 2 rounds =", m2.objVal)


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")


CPU model: 12th Gen Intel(R) Core(TM) i5-12450HX, instruction set [SSE2|AVX|AVX2]


Thread count: 12 physical cores, 12 logical processors, using up to 12 threads


Optimize a model with 57 rows, 48 columns and 224 nonzeros


Model fingerprint: 0xf3d0f635


...

Thread count was 12 (of 12 available processors)


Solution count 3: 4 6 16 


Optimal solution found (tolerance 1.00e-04)


Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%



=== 2-round MILP Results (tight MDS) ===

--- Round 0 ---
x_in[0,0]   = 0
x_in[0,1]   = 0
x_in[0,2]   = 0
x_in[0,3]   = 0
x_in[0,4]   = 0
x_in[0,5]   = 0
x_in[0,6]   = 0
x_in[0,7]   = 1
x_sbox[0,0] = 0
x_sbox[0,1] = 0
x_sbox[0,2] = 0
x_sbox[0,3] = 0
x_sbox[0,4] = 0
x_sbox[0,5] = 0
x_sbox[0,6] = 0
x_sbox[0,7] = 1
x_mds[0,0]  = 0
x_mds[0,1]  = 1
x_mds[0,2]  = 0
x_mds[0,3]  = 0
x_mds[0,4]  = 0
x_mds[0,5]  = 1
x_mds[0,6]  = 1
x_mds[0,7]  = 0

--- Round 1 ---
x_in[1,0]   = 0
x_in[1,1]   = 1
x_in[1,2]   = 0
x_in[1,3]   = 0
x_in[1,4]   = 0
x_in[1,5]   = 1
x_in[1,6]   = 1
x_in[1,7]   = 0
x_sbox[1,0] = 0
x_sbox[1,1] = 1
x_sbox[1,2] = 0
x_sbox[1,3] = 0
x_sbox[1,4] = 0
x_sbox[1,5] = 1
x_sbox[1,6] = 1
x_sbox[1,7] = 0
x_mds[1,0]  = 1
x_mds[1,1]  = 1
x_mds[1,2]  = 1
x_mds[1,3]  = 1
x_mds[1,4]  = 1
x_mds[1,5]  = 1
x_mds[1,6]  = 1
x_mds[1,7]  = 1

Total active S-boxes over 2 rounds = 4.0
